This Query is created by user whiteklay

In [2]:
%%hive_spark
WITH intermediaries AS (
    SELECT 
        PCOM_CUSTOMER.CUST_CODE AS intermediary_code,
        PCOM_CUSTOMER.CUST_NAME AS intermediary_name,
        CASE 
            WHEN DATEDIFF(DAY, vo_login.aml_acknowledged_timestamp, GETDATE()) >= 365 THEN 'Yes'
            ELSE 'No'
        END AS due_aml_training,
        'D-1' AS reported_date,
        GETDATE() AS load_timestamp
    FROM PCOM_CUSTOMER
    LEFT JOIN T_RPDB_POL_PARTY 
        ON LEFT(PCOM_CUSTOMER.CUST_CODE, 2) = T_RPDB_POL_PARTY.cust_code
    LEFT JOIN vo_login 
        ON T_RPDB_POL_PARTY.party_code = vo_login.is_aml_acknowledged
),

fulfillers AS (
    SELECT 
        MAX(agg_policy_master_report_daily.policy_from_date) AS max_policy_from_date
    FROM agg_policy_master_report_daily
    WHERE intermediary_code = agg_policy_master_report_daily.fufiller_code
)

SELECT 
    intermediaries.intermediary_code,
    intermediaries.intermediary_name,
    intermediaries.due_aml_training,
    intermediaries.reported_date,
    intermediaries.load_timestamp,
    agg_policy_master_report_daily.fufiller_code,
    agg_policy_master_report_daily.fulfiller_name,
    agg_policy_master_report_daily.office_code,
    agg_policy_master_report_daily.office_name,
    agg_policy_master_report_daily.zone_name,
    agg_policy_master_report_daily.mkt_zone_name
FROM intermediaries
LEFT JOIN agg_policy_master_report_daily 
    ON intermediaries.intermediary_code = agg_policy_master_report_daily.fufiller_code
    AND intermediaries.max_policy_from_date = agg_policy_master_report_daily.policy_from_date